# Import Modules

In [8]:
import h2o
import pandas

In [9]:
project_path = "/gtc-2017"

# Connect or Start H2O

In [10]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,3 hours 12 mins
H2O cluster version:,3.11.0.226
H2O cluster version age:,2 days
H2O cluster name:,root
H2O cluster total nodes:,1
H2O cluster free memory:,25.70 Gb
H2O cluster total cores:,20
H2O cluster allowed cores:,20
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


# Import Data

Import data into H2O.

In [11]:
mnist_training = h2o.import_file(project_path+"/data/mnist-training.csv")
mnist_testing = h2o.import_file(project_path+"/data/mnist-testing.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Change the "label" column to a factor (i.e. categorical feature).

In [12]:
mnist_training["label"] = mnist_training["label"].asfactor()
mnist_testing["label"] = mnist_testing["label"].asfactor()

# Explore Data

In [13]:
mnist_training.head()

uri,label
/gtc-2017/data/mnist_png/training/6/6453.png,6
/gtc-2017/data/mnist_png/training/6/13970.png,6
/gtc-2017/data/mnist_png/training/6/21967.png,6
/gtc-2017/data/mnist_png/training/6/11277.png,6
/gtc-2017/data/mnist_png/training/6/27350.png,6
/gtc-2017/data/mnist_png/training/6/32119.png,6
/gtc-2017/data/mnist_png/training/6/44201.png,6
/gtc-2017/data/mnist_png/training/6/58770.png,6
/gtc-2017/data/mnist_png/training/6/39440.png,6
/gtc-2017/data/mnist_png/training/6/41988.png,6


# Test GPU Driver

In [14]:
!nvidia-smi

Mon May  8 22:09:44 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 378.13                 Driver Version: 378.13                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 0000:02:00.0      On |                  N/A |
| 28%   38C    P8    11W / 180W |   1186MiB /  8112MiB |     18%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Build Deep Water Model

In [15]:
from h2o.estimators.deepwater import H2ODeepWaterEstimator
model_mnist_lenet_mx = H2ODeepWaterEstimator(epochs=80, network="lenet")
model_mnist_lenet_mx.train(x=["uri"], y="label",
                           training_frame=mnist_training,
                           validation_frame=mnist_testing,
                           model_id="model_mnist_lenet_mx")

deepwater Model Build progress: |█████████████████████████████████████████| 100%


In [16]:
model_mnist_lenet_mx.show()

Model Details
H2ODeepWaterEstimator :  Deep Water
Model Key:  model_mnist_lenet_mx


ModelMetricsMultinomial: deepwater
** Reported on train data. **

MSE: 0.00326275090552
RMSE: 0.057120494619
LogLoss: 0.0154373552434
Mean Per-Class Error: 0.0040814230845
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
965.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0010352,1 / 966
0.0,1158.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0017241,"2 / 1,160"
0.0,1.0,951.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0062696,6 / 957
0.0,0.0,1.0,1030.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0038685,"4 / 1,034"
0.0,1.0,0.0,0.0,926.0,0.0,0.0,0.0,0.0,1.0,0.0021552,2 / 928
1.0,0.0,0.0,1.0,0.0,913.0,2.0,0.0,1.0,1.0,0.0065288,6 / 919
1.0,0.0,1.0,0.0,0.0,1.0,989.0,0.0,0.0,0.0,0.0030242,3 / 992
0.0,1.0,3.0,0.0,1.0,0.0,0.0,1012.0,1.0,2.0,0.0078431,"8 / 1,020"
0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,966.0,1.0,0.0041237,4 / 970
0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,939.0,0.0042418,4 / 943


Top-10 Hit Ratios: 


k,hit_ratio
1,0.9959551
2,0.9991910
3,0.9997978
4,0.9998989
5,0.9998989
6,1.0
7,1.0
8,1.0
9,1.0
10,1.0



ModelMetricsMultinomial: deepwater
** Reported on validation data. **

MSE: 0.01154996938
RMSE: 0.107470783844
LogLoss: 0.0574998806957
Mean Per-Class Error: 0.0135046949352
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
969.0,1.0,2.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,0.0112245,11 / 980
0.0,1127.0,1.0,1.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0070485,"8 / 1,135"
1.0,2.0,1024.0,2.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0077519,"8 / 1,032"
0.0,1.0,1.0,994.0,0.0,7.0,0.0,3.0,3.0,1.0,0.0158416,"16 / 1,010"
0.0,1.0,1.0,0.0,969.0,0.0,2.0,0.0,3.0,6.0,0.0132383,13 / 982
1.0,0.0,0.0,8.0,0.0,878.0,2.0,0.0,2.0,1.0,0.0156951,14 / 892
3.0,4.0,0.0,1.0,2.0,2.0,944.0,0.0,1.0,1.0,0.0146138,14 / 958
0.0,4.0,10.0,0.0,0.0,1.0,0.0,1009.0,1.0,3.0,0.0184825,"19 / 1,028"
1.0,1.0,1.0,3.0,0.0,3.0,0.0,2.0,962.0,1.0,0.0123203,12 / 974
0.0,4.0,1.0,0.0,5.0,4.0,1.0,2.0,2.0,990.0,0.0188305,"19 / 1,009"


Top-10 Hit Ratios: 


k,hit_ratio
1,0.9866
2,0.9974
3,0.9993
4,0.9996
5,0.9999
6,1.0
7,1.0
8,1.0
9,1.0
10,1.0


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
,2017-05-08 22:09:45,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan
,2017-05-08 22:09:45,0.925 sec,11906 obs/sec,0.0170667,1,1024.0,0.5367132,2.3191285,0.3155021,0.5227956,2.1446632,0.2973
,2017-05-08 22:09:53,8.805 sec,17782 obs/sec,2.2186667,130,133120.0,0.1093672,0.0516970,0.0140560,0.1270190,0.0809966,0.0187
,2017-05-08 22:09:58,13.822 sec,17979 obs/sec,3.6010667,211,216064.0,0.0796985,0.0247293,0.0075842,0.1184238,0.0634008,0.0163
,2017-05-08 22:10:03,18.839 sec,17779 obs/sec,4.9152,288,294912.0,0.0571205,0.0154374,0.0040449,0.1074708,0.0574999,0.0134
,2017-05-08 22:10:08,23.973 sec,17544 obs/sec,6.1952,363,371712.0,0.0495614,0.0097652,0.0029326,0.1070418,0.0589762,0.0134
,2017-05-08 22:10:13,29.304 sec,17145 obs/sec,7.4410667,436,446464.0,0.0356285,0.0046618,0.0015168,0.1031050,0.0584697,0.0125
,2017-05-08 22:10:18,34.382 sec,16955 obs/sec,8.6528,507,519168.0,0.0358219,0.0051723,0.0017191,0.1021673,0.0658379,0.0119
,2017-05-08 22:10:24,39.432 sec,16914 obs/sec,9.9157333,581,594944.0,0.0191815,0.0013346,0.0005056,0.1024788,0.0680970,0.0116
,2017-05-08 22:10:29,44.459 sec,16829 obs/sec,11.1445333,653,668672.0,0.0152901,0.0009157,0.0003034,0.1024734,0.0684364,0.0117


# Extract Deep Features from Model

In [17]:
extracted_features = model_mnist_lenet_mx.deepfeatures(mnist_training, "flatten0_output")

deepfeatures progress: |██████████████████████████████████████████████████| 100%


Explore the dimensionality of the deep feature and view a few samples.

In [18]:
extracted_features.dim

[60000, 800]

In [19]:
print(extracted_features[0:4,:])

DF.flatten0_output.C1 DF.flatten0_output.C2 DF.flatten0_output.C3 DF.flatten0_output.C4 DF.flatten0_output.C5 DF.flatten0_output.C6 DF.flatten0_output.C7 DF.flatten0_output.C8 DF.flatten0_output.C9 DF.flatten0_output.C10 DF.flatten0_output.C11 DF.flatten0_output.C12 DF.flatten0_output.C13 DF.flatten0_output.C14 DF.flatten0_output.C15 DF.flatten0_output.C16 DF.flatten0_output.C17 DF.flatten0_output.C18 DF.flatten0_output.C19 DF.flatten0_output.C20 DF.flatten0_output.C21 DF.flatten0_output.C22 DF.flatten0_output.C23 DF.flatten0_output.C24 DF.flatten0_output.C25 DF.flatten0_output.C26 DF.flatten0_output.C27 DF.flatten0_output.C28 DF.flatten0_output.C29 DF.flatten0_output.C30 DF.flatten0_output.C31 DF.flatten0_output.C32 DF.flatten0_output.C33 DF.flatten0_output.C34 DF.flatten0_output.C35 DF.flatten0_output.C36 DF.flatten0_output.C37 DF.flatten0_output.C38 DF.flatten0_output.C39 DF.flatten0_output.C40 DF.flatten0_output.C41 DF.flatten0_output.C42 DF.flatten0_output.C43 DF.flatten0_output.C44 DF.flatten0_output.C45 DF.flatten0_output.C46 DF.flatten0_output.C47 DF.flatten0_output.C48 DF.flatten0_output.C49 DF.flatten0_output.C50 DF.flatten0_output.C51 DF.flatten0_output.C52 DF.flatten0_output.C53 DF.flatten0_output.C54 DF.flatten0_output.C55 DF.flatten0_output.C56 DF.flatten0_output.C57 DF.flatten0_output.C58 DF.flatten0_output.C59 DF.flatten0_output.C60 DF.flatten0_output.C61 DF.flatten0_output.C62 DF.flatten0_output.C63 DF.flatten0_output.C64 DF.flatten0_output.C65 DF.flatten0_output.C66 DF.flatten0_output.C67 DF.flatten0_output.C68 DF.flatten0_output.C69 DF.flatten0_output.C70 DF.flatten0_output.C71 DF.flatten0_output.C72 DF.flatten0_output.C73 DF.flatten0_output.C74 DF.flatten0_output.C75 DF.flatten0_output.C76 DF.flatten0_output.C77 DF.flatten0_output.C78 DF.flatten0_output.C79 DF.flatten0_output.C80 DF.flatten0_output.C81 DF.flatten0_output.C82 DF.flatten0_output.C83 DF.flatten0_output.C84 DF.flatten0_output.C85 DF.flatten0_output.C86 DF.flatten0_output.C87 DF.flatten0_output.C88 DF.flatten0_output.C89 DF.flatten0_output.C90 DF.flatten0_output.C91 DF.flatten0_output.C92 DF.flatten0_output.C93 DF.flatten0_output.C94 DF.flatten0_output.C95 DF.flatten0_output.C96 DF.flatten0_output.C97 DF.flatten0_output.C98 DF.flatten0_output.C99 DF.flatten0_output.C100 DF.flatten0_output.C101 DF.flatten0_output.C102 DF.flatten0_output.C103 DF.flatten0_output.C104 DF.flatten0_output.C105 DF.flatten0_output.C106 DF.flatten0_output.C107 DF.flatten0_output.C108 DF.flatten0_output.C109 DF.flatten0_output.C110 DF.flatten0_output.C111 DF.flatten0_output.C112 DF.flatten0_output.C113 DF.flatten0_output.C114 DF.flatten0_output.C115 DF.flatten0_output.C116 DF.flatten0_output.C117 DF.flatten0_output.C118 DF.flatten0_output.C119 DF.flatten0_output.C120 DF.flatten0_output.C121 DF.flatten0_output.C122 DF.flatten0_output.C123 DF.flatten0_output.C124 DF.flatten0_output.C125 DF.flatten0_output.C126 DF.flatten0_output.C127 DF.flatten0_output.C128 DF.flatten0_output.C129 DF.flatten0_output.C130 DF.flatten0_output.C131 DF.flatten0_output.C132 DF.flatten0_output.C133 DF.flatten0_output.C134 DF.flatten0_output.C135 DF.flatten0_output.C136 DF.flatten0_output.C137 DF.flatten0_output.C138 DF.flatten0_output.C139 DF.flatten0_output.C140 DF.flatten0_output.C141 DF.flatten0_output.C142 DF.flatten0_output.C143 DF.flatten0_output.C144 DF.flatten0_output.C145 DF.flatten0_output.C146 DF.flatten0_output.C147 DF.flatten0_output.C148 DF.flatten0_output.C149 DF.flatten0_output.C150 DF.flatten0_output.C151 DF.flatten0_output.C152 DF.flatten0_output.C153 DF.flatten0_output.C154 DF.flatten0_output.C155 DF.flatten0_output.C156 DF.flatten0_output.C157 DF.flatten0_output.C158 DF.flatten0_output.C159 DF.flatten0_output.C160 DF.flatten0_output.C161 DF.flatten0_output.C162 DF.flatten0_output.C163 DF.flatten0_output.C164 DF.flatten0_output.C165 DF.flatten0_output.C166 DF.flatten0_output.C167 DF.flatten0_output.C168 DF.flatten0_output.C169 DF.flatten0_output.C170 DF.flatten0_output.C171 DF.f

# Build New Training Set with Deep Features

In [20]:
extracted_features["label"] = mnist_training["label"]

In [21]:
deep_train, deep_valid = extracted_features.split_frame(ratios=[0.8],
                                                        destination_frames=["deep_train", "deep_valid"])

# Build GBM Model Using Deep Features

In [22]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
gbm_mnist_lenet_mx = H2OGradientBoostingEstimator(ntrees=80)
gbm_mnist_lenet_mx.train(x=[x for x in extracted_features.columns if x != "label"],
                         y="label",
                         training_frame=deep_train,
                         validation_frame=deep_valid,
                         model_id="gbm_mnist_lenet_mx")

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [23]:
gbm_mnist_lenet_mx.show()

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  gbm_mnist_lenet_mx


ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.00182932649943
RMSE: 0.0427706265962
LogLoss: 0.0133472131194
Mean Per-Class Error: 0.00025389435614
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
4737.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"0 / 4,737"
0.0,5359.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"0 / 5,359"
0.0,0.0,4752.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0004207,"2 / 4,754"
0.0,0.0,0.0,4920.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0002032,"1 / 4,921"
1.0,0.0,0.0,0.0,4690.0,0.0,0.0,1.0,0.0,1.0,0.0006392,"3 / 4,693"
0.0,0.0,0.0,0.0,0.0,4333.0,0.0,0.0,0.0,1.0,0.0002307,"1 / 4,334"
0.0,0.0,0.0,0.0,1.0,0.0,4710.0,0.0,1.0,0.0,0.0004244,"2 / 4,712"
0.0,0.0,0.0,0.0,0.0,0.0,0.0,5009.0,0.0,1.0,0.0001996,"1 / 5,010"
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4705.0,1.0,0.0002125,"1 / 4,706"
1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4795.0,0.0002085,"1 / 4,796"


Top-10 Hit Ratios: 


k,hit_ratio
1,0.9997501
2,0.9999584
3,0.9999792
4,1.0
5,1.0
6,1.0
7,1.0
8,1.0
9,1.0
10,1.0



ModelMetricsMultinomial: gbm
** Reported on validation data. **

MSE: 0.0187968155173
RMSE: 0.137101478903
LogLoss: 0.0667450419701
Mean Per-Class Error: 0.0200424205729
Confusion Matrix: vertical: actual; across: predicted



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
,2017-05-08 22:12:59,0.007 sec,0.0,0.9,2.3025851,0.8977760,0.9,2.3025851,0.8991484
,2017-05-08 22:13:01,2.936 sec,1.0,0.7945552,1.6055037,0.0871892,0.7971273,1.6216316,0.1044415
,2017-05-08 22:13:06,7.760 sec,3.0,0.6459707,1.0762429,0.0599517,0.6535351,1.1054574,0.0748873
,2017-05-08 22:13:11,12.655 sec,5.0,0.5298114,0.7884538,0.0503727,0.5425098,0.8250771,0.0651194
,2017-05-08 22:13:16,17.358 sec,7.0,0.4396791,0.5994535,0.0420432,0.4569591,0.6393392,0.0561863
---,---,---,---,---,---,---,---,---,---
,2017-05-08 22:15:53,2 min 54.754 sec,72.0,0.0505087,0.0163734,0.0005622,0.1402277,0.0696885,0.0197028
,2017-05-08 22:15:58,2 min 59.608 sec,74.0,0.0485270,0.0155832,0.0004373,0.1393672,0.0688993,0.0198698
,2017-05-08 22:16:03,3 min 4.675 sec,76.0,0.0467284,0.0148278,0.0004165,0.1385909,0.0681963,0.0198698
,2017-05-08 22:16:08,3 min 9.749 sec,78.0,0.0447870,0.0140991,0.0003124,0.1378477,0.0673930,0.0197863



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
DF.flatten0_output.C88,10036.6826172,1.0,0.0616319
DF.flatten0_output.C620,8859.3681641,0.8826988,0.0544024
DF.flatten0_output.C558,8159.4116211,0.8129590,0.0501042
DF.flatten0_output.C488,8102.7915039,0.8073177,0.0497565
DF.flatten0_output.C141,7918.9619141,0.7890019,0.0486277
---,---,---,---
DF.flatten0_output.C764,0.0,0.0,0.0
DF.flatten0_output.C766,0.0,0.0,0.0
DF.flatten0_output.C768,0.0,0.0,0.0
DF.flatten0_output.C799,0.0,0.0,0.0



See the whole table with table.as_data_frame()
